In [1]:
from veneer.manage import start, create_command_line, kill_all_now
import veneer
import pandas as pd
%matplotlib inline
%xmode Verbose
import gc 

Exception reporting mode: Verbose


In [2]:
# I will put this here just for reference, this cell can be run to shutdown all of the veneer instatiated source models
#kill_all_now(processes)

Read in some data sets

In [3]:
plane_creek = pd.read_csv('Plane_Creek.csv')
plane_creek_subcatchments = plane_creek['sub_catchments'].tolist()
plane_creek_links = plane_creek['links'].tolist()

## Setup veneer

In [4]:
## define paths to veneer command and the catchment project
#path = 'C:\\Users\\bennettf\\Desktop\\vcmd\\FlowMatters.Source.VeneerCmd.exe'
##catchment_project path

#Rob
veneer_install = 'D:/PythonScriptingForGBR/VeneerPlugin/v411'
#source_version = '4.1.1'
#By specifying source_path I can use my local Source compilation/code, using source_version above uses tyhe sinatlled version
source_pathstr = 'D:/SourceSource/Output'
cmd_directory = 'C:/tempveneer/veneer_cmd'
#dest = 'C:\\tempveneer\\veneer_cmd'
#path = create_command_line(veneer_install,source_version,dest=cmd_directory)
path = create_command_line(veneer_install,source_version=None,source_path=source_pathstr,dest=cmd_directory)
path
##End Rob



#catchment_project='D:/PythonScriptingForGBR/reef_scripts/FredTest/source_model/MW_RC7_Hydro.rsproj'
catchment_project='P:/projects/RC8_ResultsSets/MW/MW_BASE_RC8_Rob.rsproj'

# Number of instances to open
num_copies=1    # Important - set this to be a number ~ the number of CPU cores in your system!
first_port=9950

In [5]:
#Now, go ahead and start source
processes,ports = start(catchment_project,
                        n_instances=num_copies,
                        ports=first_port,
                        debug=True,
                        veneer_exe=path,
                        remote=False )

Starting C:\tempveneer\veneer_cmd\FlowMatters.Source.VeneerCmd.exe -p 9950 -s  P:\projects\RC8_ResultsSets\MW\MW_BASE_RC8_Rob.rsproj
[0] Loading plugins

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\DERMTools.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\Dynamic_SedNet.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\GBR_DynSed_Extension.dll

[0] Loaded D:\PythonScriptingForGBR\VeneerPlugin\v411\FlowMatters.Source.Veneer.dll

[0] Plugins loaded (4/4)

[0] Opening project file: P:\projects\RC8_ResultsSets\MW\MW_BASE_RC8_Rob.rsproj

ERROR[0] log4net:ERROR Failed to find configuration section 'log4net' in the application's .config file. Check your .config file for the <log4net> and <configSections> elements. The configuration section should look like: <section name="log4net" type="log4net.Config.Log4NetConfigurationSectionHandler,log4net" />

[0] Loading project

[0] Project Loaded

[0] MW_DS_2015

[0] MW_DS_2015

[0] [3:28:05 PM] Venee

Assign veneer clients

In [6]:
vs = [veneer.Veneer(port=p) for p in ports]
v = vs[0]

Ok, we have the list, now define the ones we want to investigate

Setup details to write parameters and read outputs 

In [7]:
current_node = 'gauge_126002A_PlaneCkSarina' 
parameters = pd.read_csv('parameters.csv')
v.configure_recording(disable=[{}],enable=[{'RecordingVariable':'Downstream Flow Volume', 'NetworkElement':current_node}])
v.configure_recording(enable=[{'RecordingVariable':'Outflow', 'NetworkElement':'SC #160'}])
v.configure_recording(enable=[{'RecordingVariable':'Slow Flow', 'NetworkElement':'SC #160'}])


Ok, that should do it, let's go

In [ ]:
#names = list(parameters.columns.values)
names = ['Uzk','Uztwm', 'Uzfwm','Rexp','Lztwm']
num_params = len(names)
for index, row in parameters.iterrows():
    
    # Start iteration with a bit of house keeping
    collected = gc.collect()
    print("Garbage collector: collected %d objects." % (collected))
    
    current_set = str(int(row['Set']))
    current_params_water = []
    current_params_conservation = []
    for name in names:
        current_params_water.append(row[name+'_1'])
        current_params_conservation.append(row[name+'_2'])
    print('current set : ', current_set)
    print('current set water : ', current_params_water)
    print('current set conservation : ', current_params_conservation)
    print('==============================================')
    
    for i in range(0,num_params):
        v.model.catchment.runoff.set_param_values(names[i], current_params_water[i] ,catchments = plane_creek_subcatchments, fus = 'Water')
        v.model.catchment.runoff.set_param_values(names[i], current_params_conservation[i] ,catchments = plane_creek_subcatchments, fus = 'Conservation')
    #run model
    v.run_model(params={'NoHardCopyResults':True},start="01/07/1986",end="30/06/1988") 
    
    output_file_name = 'ds_' + current_set + '.csv'
    current_time_series = v.retrieve_multiple_time_series(criteria={'RecordingVariable':'Downstream Flow Volume','NetworkElement':current_node})
    slow_flow = v.retrieve_multiple_time_series(criteria={'RecordingVariable':'Slow Flow','NetworkElement':'SC #160'})['SC #160:Slow Flow']
    outflow = v.retrieve_multiple_time_series(criteria={'RecordingVariable':'Outflow','NetworkElement':'SC #160'})['SC #160:Outflow']
    current_time_series = current_time_series.truncate(after='1988-09-30', before='1972-10-19')
    current_time_series.to_csv('data_out/' + output_file_name)
    slow_flow.to_csv('data_out/slowflow/' + output_file_name)
    outflow.to_csv('data_out/outflow/' + output_file_name)




Garbage collector: collected 0 objects.
current set :  1
current set water :  [0.74636955500000002, 221.41435079999999, 22.796852650000002, 1.5439575569999999, 703.68778350000002]
current set conservation :  [1.2813660709999999, 101.64008879999999, 92.099758379999997, 4.3867658369999996, 486.78510520000003]
Garbage collector: collected 75 objects.
current set :  2
current set water :  [1.2292846399999999, 133.141423, 82.142497480000003, 1.5012330040000001, 773.52462329999992]
current set conservation :  [0.71524897099999996, 150.84645940000001, 56.709774580000001, 3.0432114549999998, 99.803511510000007]
